In [17]:
import numpy as np

def T_n(x, n):
    return np.cos(n * np.arccos(x))

def T_n_prime(x, n):
    return (n * np.sin(n * np.arccos(x))) / np.sqrt(1 - x**2 + 1e-12)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    s = sigmoid(x)
    return s * (1 - s)

def sigmoid_double_prime(x):
    s = sigmoid(x)
    return (1 - 2*s) * s * (1 - s)

def tanh(x):
    return np.tanh(x)

def sech2(x):
    return 1 / np.cosh(x)**2
def cheby_sigmoid(x, alpha=1.0, beta=0.5, n=2):
    u = np.tanh(beta * x)
    return sigmoid(x) + alpha * T_n(u, n) * sigmoid_prime(x)

def cheby_sigmoid_prime(x, alpha=1.0, beta=0.5, n=2):
    u = np.tanh(beta * x)
    return (sigmoid_prime(x)
            + alpha * (T_n(u, n) * sigmoid_double_prime(x)
            + sigmoid_prime(x) * T_n_prime(u, n) * beta * sech2(beta*x)))

def cheby_tanh(x, alpha=1.0, beta=0.5, n=2):
    u = np.tanh(beta * x)
    return tanh(x) + alpha * T_n(u, n) * sech2(x)

def cheby_tanh_prime(x, alpha=1.0, beta=0.5, n=2):
    u = np.tanh(beta * x)
    return (sech2(x)
            + alpha * (T_n(u, n) * (-2 * tanh(x) * sech2(x))
            + sech2(x) * T_n_prime(u, n) * beta * sech2(beta*x)))

def lipschitz_constant(func_prime, xmin=-10, xmax=10, grid=10000, **kwargs):
    xs = np.linspace(xmin, xmax, grid)
    vals = func_prime(xs, **kwargs)
    return np.max(np.abs(vals))

alpha, beta, n = 0.4, 0.9, 3

L_sigmoid_base = lipschitz_constant(sigmoid_prime)
L_tanh_base    = lipschitz_constant(sech2)

L_sigmoid_cheb = lipschitz_constant(cheby_sigmoid_prime, alpha=alpha, beta=beta, n=n)
L_tanh_cheb    = lipschitz_constant(cheby_tanh_prime, alpha=alpha, beta=beta, n=n)

print("Baseline Lipschitz constant (Sigmoid):", L_sigmoid_base)
print("Baseline Lipschitz constant (Tanh):", L_tanh_base)
print("Estimated Lipschitz constant (Cheby-Sigmoid):", L_sigmoid_cheb)
print("Estimated Lipschitz constant (Cheby-Tanh):", L_tanh_cheb)


Baseline Lipschitz constant (Sigmoid): 0.24999993748750857
Baseline Lipschitz constant (Tanh): 0.999998999800637
Estimated Lipschitz constant (Cheby-Sigmoid): 0.3318652129548819
Estimated Lipschitz constant (Cheby-Tanh): 1.0517399430302916
